# DeX Modeling

A state machine is a tuple $(S, I, O, \tau : S \times I \rightarrow S, \eta : S \times I \rightarrow O)$ where $S$ is a set of states, $I$ is a set of inputs, $O$ is a set of outputs, $\tau$ is a state transition function ($(s_t, i_t) \mapsto s_{t+1}$), and $\eta$ is the output function ($(s_t, i_t) \mapsto o_{t}$)

Let's model a dex as 
$$\text{dex} = \left( \mathbb{M}_{n,n}^{(\mathbb{R}^+ \cup \{0\}) \times \mathbb{R}^+}, 1..n \times \mathbb{R}^+ \times 1..n, \mathbb{R}^+ \times \mathbb{R}^+, \tau, \eta \right)$$
For set of $n \times n$ matrices with nonnegative real 2-tuples as components (where the second component also excludes zero) as states for number of assets $n$; a 3-tuple containing an integer id of an asset to deposit, a positive real number representing how much of it to deposit, and an integer id of an asset to withdraw as inputs; a 2-tuple of positive reals representing liquidity tokens earned on the left and representing withdraw asset withdrawn on the right as outputs; transition function $\tau$ and output function $\eta$ to be defined. 

Note: make price a function of balance.

$$\forall n \in \mathbb{N} \setminus \{0, 1\}, \forall A \in \mathbb{M}_{n,n}^{(\mathbb{R}^+ \cup \{0\}) \times \mathbb{R}^+}, \forall (i, j) \in 1..n \times 1..n, \\ 
\text{fst}\ .\ A\ (i, j)\ \text{denotes the balance of the ith asset in a pool against the jth asset, and } \\ \text{snd}\ .\ A\ (i, j)\ \text{denotes the price of the ith asset in a pool against the jth asset}.$$

This model omits **pool picking**, and the **caller** (of functions $\tau$ and $\eta$) is the liquidity provider or swapper yet we do not track the state of the caller. 

Later, black boxes $\tau_{i,j}^x$ and $\tau_{i,j}^y$ will allow us to abstract over constant-product, constant-sum, or stableswap price updating. 

I've been misleading. By $\mathbb{M}_{n,n}^{(\mathbb{R}^+ \cup \{0\}) \times \mathbb{R}^+}$ I in fact mean only matrices with $(\infty, \infty)$ down the diagonal. 

Let's profile a step of the state machine at $n=3, \forall t \in \mathbb{N}$. 

We have some function $\forall i\ j \in 1..n, LT_{i,j} : \mathbb{R}^+ \rightarrow \mathbb{R}^+$ defining the liquidity tokens you get when you deposit $a \in \mathbb{R}^+$ of asset $i$ against asset $j$, and some function $w_{i,j} : \mathbb{R}^+ \times \mathbb{R}^+ \rightarrow \mathbb{R}$ defining the amount of asset $j$ you get back if you deposit $a \in \mathbb{R}^+$ of asset $i$ when the price of $i$ against $j$ is $y_{i,j,t} \in \mathbb{R}^+$.


$$\tau 
\left( 
\begin{bmatrix} 
(\infty, \infty) & (x_{1,2}, y_{1,2}) & (x_{1,3}, y_{1,3}) \\
(x_{2,1}, y_{2,1}) & (\infty, \infty) & (x_{2,3}, y_{2,3}) \\
(x_{3,1}, y_{3,1}) & (x_{3,2}, y_{3,2}) & (\infty, \infty)
\end{bmatrix}_t,
(1, a, 2)
\right) = 
\begin{bmatrix} 
(\infty, \infty) & (x_{1,2}, y_{1,2}) & (x_{1,3}, y_{1,3}) \\
(x_{2,1}, y_{2,1}) & (\infty, \infty) & (x_{2,3}, y_{2,3}) \\
(x_{3,1}, y_{3,1}) & (x_{3,2}, y_{3,2}) & (\infty, \infty)
\end{bmatrix}_{t+1}
$$

$$
\eta 
\left(
\begin{bmatrix} 
(\infty, \infty) & (x_{1,2}, y_{1,2}) & (x_{1,3}, y_{1,3}) \\
(x_{2,1}, y_{2,1}) & (\infty, \infty) & (x_{2,3}, y_{2,3}) \\
(x_{3,1}, y_{3,1}) & (x_{3,2}, y_{3,2}) & (\infty, \infty)
\end{bmatrix}_t, 
(1, a, 2) 
\right) = 
\left(LT_{1,2}(a), w_{1,2}(a, y_{1,2,t})\right)
$$

#### "The caller deposits $a$ of asset $1$ into a pool against asset $2$, gaining $LT_{1,2}(a)$ liquidity tokens and $w(a, y_{1,2,t})$ of asset $2$"

What is not pictured is $\forall t \in \mathbb{N}, \forall i\ j \in 1..n, \exists \tau_{i,j}^x : (\mathbb{R}^+ \cup \{0\}) \times \mathbb{R}^+ \rightarrow (\mathbb{R}^+ \cup \{0\}), \exists \tau_{i,j}^y : \mathbb{R}^+ \times \mathbb{R}^+ \rightarrow \mathbb{R}^+, \tau_{i,j}^x(x_{i,j,t}, a) = x_{i,j,t+1} \land \tau_{i,j}^y(y_{i,j,t}, a) = y_{i,j,t+1}$. The subtle $t+1$ subscript on the output of $\tau$ is sufficient to show that the value of $(x_{1,2}, y_{1,2})$ changed. 

# Notes from the call

this model is missing 
1. Part of state: `isPoolKilled` in which case the only transaciton people can do is remove liquidity
2. Liquidity token **supply**
3. 

Ben is saying sentential > functional/matrix "b(x1) is the balance of some token x1. 

parameterize concepts like supply over tokens. 

[we're actually gonna use something like this](https://github.com/ArdanaLabs/DanaSwap/blob/b571cbc9eec17f144626f49646e0bf1e14351e3a/src/DanaSwap/Contract.hs#L268)

What if 

In [61]:
import numpy as np

In [62]:
class Swapper: 
    def __init__(self, ):
        pass
    
class LiquidityProvider: 
    def __init__(self, ): 
        pass
    
class DeX: 
    """constant sum rather than stableswap, suppressing pool selection concerns.
    
    NOT an implementation of the above latex. 
    """
    def __init__(self, balances: dict):
        self.reset(balances)
            
    def reset(self, balances: dict): 
        for asset_label, balance in balances.items(): 
            assert isinstance(asset_label, str)
            assert isinstance(balance, (int, float))
        self.balances = balances
        self.asset_labels = list(balances.keys())
        self.n = len(self.asset_labels)
        self.D = sum(balances.values())
        # balances.keys() is sort of like the row & col labels. 
        self.board = np.zeros((self.n, self.n, 2))
        for i, label_i in enumerate(self.asset_labels): 
            for j, label_j in enumerate(self.asset_labels): 
                self.board[i, j] = [self.D - self.balances[label_i], self.D - self.balances[label_j]]
        for i in range(self.n): 
            # You can't trade x for x
            self.board[i, i] = [np.nan, np.nan]
            
    def trade(self, asset_deposit: str, amount_deposit: float, asset_withdraw: str, amount_withdraw: float): 
        assert asset_deposit in self.balances.keys() and asset_withdraw in self.balances.keys()
        
        assert self.balances[asset_withdraw] >= amount_withdraw
        
        self.reset({})

In [63]:
# A DeX with 4 coins in it. 
dex4 = DeX({"w": 6., "x": 3., "y": 9., "z": 2.})
print(dex4.board)

dex4.trade("w", 2, "y", 3)

[[[nan nan]
  [14. 17.]
  [14. 11.]
  [14. 18.]]

 [[17. 14.]
  [nan nan]
  [17. 11.]
  [17. 18.]]

 [[11. 14.]
  [11. 17.]
  [nan nan]
  [11. 18.]]

 [[18. 14.]
  [18. 17.]
  [18. 11.]
  [nan nan]]]
